# Level 17: eat?

[http://www.pythonchallenge.com/pc/return/romance.html](http://www.pythonchallenge.com/pc/return/romance.html)

## 線索

標題是 `eat?`，頁面上有一張圖片，主要是一堆餅乾的照片，左下角有另一張圖，仔細看，是 [Level 4 - follow the chain](04_follow_the_chain.ipynb) 的那張圖。

查看 HTML Source，沒有任何其他線索。

## 嘗試

試著變更以下幾個 URL，但看起來不像是更進一步的線索：

```http://www.pythonchallenge.com/pc/return/cookies.html```

> yummy! chocolate chips. 

```http://www.pythonchallenge.com/pc/return/chocolate.html```

> do you want to eat or to play? 

```http://www.pythonchallenge.com/pc/return/play.html```

> then go back and play. 

接下來其實有好幾天沒有進展，直到偶然間我想到或許可以看看 document.cookie

In [2]:
import urllib.request

baseurl = "http://www.pythonchallenge.com/pc/return/"

pwmgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
pwmgr.add_password(None, baseurl, "huge", "file")
auth_handler = urllib.request.HTTPBasicAuthHandler(pwmgr)
opener = urllib.request.build_opener(auth_handler)
urllib.request.install_opener(opener)

In [4]:
url = "http://www.pythonchallenge.com/pc/return/romance.html"

res = urllib.request.urlopen(url)
print(res.headers)

Vary: Accept-Encoding
Content-Type: text/html
Accept-Ranges: bytes
ETag: "3716707270"
Last-Modified: Sat, 12 Mar 2016 19:38:46 GMT
Content-Length: 207
Connection: close
Date: Mon, 05 Feb 2024 09:19:38 GMT
Server: lighttpd/1.4.55




有可能的網址都試一試

In [5]:
urls = [
    "http://www.pythonchallenge.com/pc/return/cookies.html",
    "http://www.pythonchallenge.com/pc/return/chocolate.html",
    "http://www.pythonchallenge.com/pc/return/play.html"
]

for url in urls:
    res = urllib.request.urlopen(url)
    print(res.headers)
    print(res.read().decode("utf-8"))

Content-Type: text/html
Accept-Ranges: bytes
ETag: "162111112"
Last-Modified: Sat, 12 Mar 2016 19:38:46 GMT
Content-Length: 24
Connection: close
Date: Mon, 05 Feb 2024 09:21:58 GMT
Server: lighttpd/1.4.55


yummy! chocolate chips.

Content-Type: text/html
Accept-Ranges: bytes
ETag: "2309496474"
Last-Modified: Sat, 12 Mar 2016 19:38:46 GMT
Content-Length: 31
Connection: close
Date: Mon, 05 Feb 2024 09:21:58 GMT
Server: lighttpd/1.4.55


do you want to eat or to play?

Content-Type: text/html
Accept-Ranges: bytes
ETag: "1009340574"
Last-Modified: Sat, 12 Mar 2016 19:38:46 GMT
Content-Length: 23
Connection: close
Date: Mon, 05 Feb 2024 09:21:58 GMT
Server: lighttpd/1.4.55


then go back and play.



第四關 follow the chain 的網址也試一下（其實我是又過了好久之後，才想到這個也可以試一下）

In [6]:
url = "http://www.pythonchallenge.com/pc/def/linkedlist.php"

res = urllib.request.urlopen(url)
print(res.headers)

Set-Cookie: info=you%20should%20have%20followed%20busynothing...; expires=Mon, 12-Feb-2024 09:23:06 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
Content-type: text/html; charset=UTF-8
Content-Length: 524
Connection: close
Date: Mon, 05 Feb 2024 09:23:06 GMT
Server: lighttpd/1.4.55




有！東！西！了！

In [7]:
from http.cookies import SimpleCookie
import urllib.parse

cookie = SimpleCookie()
cookie.load(res.headers['Set-Cookie'])

print(urllib.parse.unquote(cookie['info'].value))

you should have followed busynothing...


我想它的意思應該是要我們做像是 follow the chain 那樣的操作，只是把 nothing 改為 busynothing  
但它沒說一開始的 busynothing 要設定為什麼，先用空白試試看，一樣迴圈跑 400 次

In [8]:
baseurl = "http://www.pythonchallenge.com/pc/def/linkedlist.php?busynothing="
nothing = ""

for i in range(400):
    res = urllib.request.urlopen(baseurl + nothing)
    content = res.read().decode()
    cookies = res.headers['Set-Cookie']
    print(content, cookies)
    nothing = content.split()[-1]
    if not nothing.isnumeric():
        break

and the next busynothing is 55274 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 93397 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 78768 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 82003 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 85014 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 54090 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 96665 info=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.pythonchallenge.com
and the next busynothing is 91178 info=deleted; expires=Thu, 0

400次嘗試的結果，Cookie 中的 info 值都是 deleted，蠻奇怪的

或許我應該跟 follow the chain 一樣，從 12345 開始試試看

In [9]:
baseurl = "http://www.pythonchallenge.com/pc/def/linkedlist.php?busynothing="
nothing = "12345"

for i in range(400):
    res = urllib.request.urlopen(baseurl + nothing)
    content = res.read().decode()
    cookies = res.headers['Set-Cookie']
    print(content, cookies)
    nothing = content.split()[-1]
    if not nothing.isnumeric():
        break

If you came here from level 4 - go back!<br>You should follow the obvious chain...<br><br>and the next busynothing is 44827 info=B; expires=Mon, 12-Feb-2024 09:31:07 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
and the next busynothing is 45439 info=Z; expires=Mon, 12-Feb-2024 09:31:07 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
and the next busynothing is 94485 info=h; expires=Mon, 12-Feb-2024 09:31:07 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
and the next busynothing is 72198 info=9; expires=Mon, 12-Feb-2024 09:31:07 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
and the next busynothing is 80992 info=1; expires=Mon, 12-Feb-2024 09:31:08 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
and the next busynothing is 8880 info=A; expires=Mon, 12-Feb-2024 09:31:08 GMT; Max-Age=604800; path=/; domain=.pythonchallenge.com
and the next busynothing is 40961 info=Y; expires=Mon, 12-Feb-2024 09:31:08 GMT; Max-Age=604800; path=/; domain=.

Wow! 出現了不一樣的結果！

info 的資訊看起來很像某種出現過的東西，我們先將所有的 info 值收集起來看看：

In [10]:
baseurl = "http://www.pythonchallenge.com/pc/def/linkedlist.php?busynothing="
nothing = "12345"

data = ""

for i in range(400):
    res = urllib.request.urlopen(baseurl + nothing)
    content = res.read().decode()
    cookies = res.headers["Set-Cookie"]
    # print(content, cookies)
    cookie = SimpleCookie()
    cookie.load(cookies)
    data += cookie["info"].value
    nothing = content.split()[-1]
    if not nothing.isnumeric():
        break

print(data)

BZh91AY%26SY%94%3A%E2I%00%00%21%19%80P%81%11%00%AFg%9E%A0%20%00hE%3DM%B5%23%D0%D4%D1%E2%8D%06%A9%FA%26S%D4%D3%21%A1%EAi7h%9B%9A%2B%BF%60%22%C5WX%E1%ADL%80%E8V%3C%C6%A8%DBH%2632%18%A8x%01%08%21%8DS%0B%C8%AF%96KO%CA2%B0%F1%BD%1Du%A0%86%05%92s%B0%92%C4Bc%F1w%24S%85%09%09C%AE%24%90


像是前面有出現過的 bz2，對吧？

In [13]:
import bz2

data_unquote = urllib.parse.unquote_to_bytes(data)
print(bz2.decompress(data_unquote).decode())

is it the 26th already? call his father and inform him that "the flowers are on their way". he'll understand.


這訊息量太大了點吧 XD

看起來要先[查查 Wolfgang Amadeus Mozart 他爸是誰](https://www.google.com/search?q=mozart%27s+father)  
然後用 [Level 13 - call him](13_call_him.ipynb) 的方式打電話給他

Amadeus 他老爸叫做 Leopold

In [14]:
import xmlrpc.client

x = xmlrpc.client.ServerProxy("http://www.pythonchallenge.com/pc/phonebook.php")
print(x.phone("Leopold"))


555-VIOLIN


試看看照 Level 13 的方式組合網址：

```http://www.pythonchallenge.com/pc/return/violin.html```

結果得到

> no! i mean yes! but ../stuff/violin.php. 

意思應該是要我們改成：

```http://www.pythonchallenge.com/pc/stuff/violin.php```

成功進入一個新的頁面，但頁面上只有一張 Leopold 的照片，網頁標題寫著 “it's me. what do you want?”

In [15]:
print(urllib.request.urlopen("http://www.pythonchallenge.com/pc/stuff/violin.php").read().decode())

<html>
<head>
  <title>it's me. what do you want?</title>
  <link rel="stylesheet" type="text/css" href="../style.css">
</head>
<body>
	<br><br>
	<center><font color="gold">
	<img src="leopold.jpg" border="0"/>
<br><br>
</font>
</body>
</html>



回頭看前兩步，提示告訴我們說：  
call his father and *inform him that "the flowers are on their way"*.

In [16]:
headers = {"Cookie": "info=" + urllib.parse.quote("the flowers are on their way")}
req = urllib.request.Request("http://www.pythonchallenge.com/pc/stuff/violin.php", headers=headers)
res = urllib.request.urlopen(req)
print(res.read().decode())

<html>
<head>
  <title>it's me. what do you want?</title>
  <link rel="stylesheet" type="text/css" href="../style.css">
</head>
<body>
	<br><br>
	<center><font color="gold">
	<img src="leopold.jpg" border="0"/>
<br><br>
oh well, don't you dare to forget the balloons.</font>
</body>
</html>



測試從上面得到的幾個關鍵字，將他們組成 URL 試試看：

```
http://www.pythonchallenge.com/pc/stuff/dare.php
http://www.pythonchallenge.com/pc/stuff/forget.php
http://www.pythonchallenge.com/pc/stuff/balloons.php
http://www.pythonchallenge.com/pc/stuff/dare.html
http://www.pythonchallenge.com/pc/stuff/forget.html
http://www.pythonchallenge.com/pc/stuff/balloons.html
```

前五組都會得到 404 Not Found  
但最後一組會導向到 ```http://www.pythonchallenge.com/pc/return/balloons.html``` 而進入下一關

是說這一題的規模也太龐大了吧！